<a href="https://colab.research.google.com/github/spencerleewilliams/cse380-notebooks/blob/master/08_3_About_Why_RSA_Works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About Why RSA Works
## Class Directed Learning
### Due: Tuesday, 23 February 2021, 11:59 pm

## The Reason

The reason RSA works is that the encryption and decryption operations are "inverses" of each other. But why does that make it work?

## Repeat Chat Post

Clarification: In the inside chat, Kyle pointed out an error in what I said while going over the reading quiz, specifically regarding Question 6, which states: If $m$ is a factor of $a$, and $n$ is a factor of $a$, then $mn$ is a factor of $a$.

Restated: If $m | a$ and $n | a$ then does $mn | a$ for any integers $m, n$?

Convince yourself that the answer is no. I said if $m$ and $n$ are both prime, then the answer is yes. Kyle gave the example of $m = 5$ and $n = 5$, both $5$ and $5$ divide $15$, but $25$ doesn't.

But the correct answer is, if m and n are **coprime** (which excludes them from being the same prime) then it is a true statement. And if $m$ and $n$ are different primes they are automatically coprime.

So, for coprime $m$ and $n$, if $m | a$ and $n | a$, then $mn | a$. For example, $8$ and $9$ are coprime (though neither is prime), and $8 | 144, 9 | 144,$ and $72 | 144$ also.

What does this have to do with the Chinese Remainder Theorem guaranteeing that, for example, with $p$, $q$ and $s$ being different primes, if $x \equiv_{p} r$ and $x \equiv_{q} r$ and $x \equiv_{s} r$, then $x \equiv_{pqs} r$?

If you need help visualizing how simultaneous congruences work (read "[=]" as $\equiv$):


In [ ]:
def format_congruence(x, m, r):
  d = x - r
  return f'{x} [=] {r} (mod {m}) -> {m} | {x} - {r} (= {d}) -> {d} = {m} * k where k = {d // m}.'

x_list = [8, 8, 8]
m_list = [3, 4, 5]
r_list = [2, 0, 3]
for f in map(format_congruence, x_list, m_list, r_list):
  print(f, '\n')

x_list = [16, 16]
m_list = [3, 11]
r_list = [1, 5]
for f in map(format_congruence, x_list, m_list, r_list):
  print(f, '\n')

## TODO Study and Learn

A proof that shows why RSA works goes as follows:

We chose two different primes $p$ and $q$, and because we chose $e$ such that for $t = (p - 1)(q - 1)$, gcd($e$, $t$) $= 1$, an inverse of $e$ mod $t$ exists and can be found. This inverse is the decryption key $d$.

Knowing $d$ and $c$ (the ciphertext), $m$ (the plaintext message) can be recovered by noting that if

$de \equiv_{t} 1$, then there is an integer $k$ such that

$de = 1 + k(p - 1)(q - 1)$.

So we know that $c^d \equiv_{n} (m^e)^d$.

But then $(m^e)^d = m^{ed} = m^{de} = m^{1 + k(p - 1)(q - 1)}$.

Enter Fermat's Little Theorem: If $r$ is prime and does not divide integer $a$, then $a^{r-1} \equiv_{r} 1$.

That means $m^{p-1} \equiv_{p} 1$, and $m^{q-1} \equiv_{q} 1$.

Combining those two facts with the above (continued)

$c^d \equiv_{n} (m^e)^d = m^{ed} = m^{de} = m^{1 + k(p - 1)(q - 1)} = m^1 \cdot m^{k(p - 1)(q - 1)}$

$= m^1 \cdot m^{(p - 1)k(q - 1)} = m^1 \cdot m^{(q - 1)k(p - 1)}$.

Because $x \equiv_{rs} y \rightarrow x \equiv_{r} y$ and $x \equiv_{s} y$, we know that

$c^d \equiv_{p} m^1 \cdot (m^{p - 1})^{k(q - 1)} \equiv_{p} m \cdot (1)^{k(q - 1)} \equiv_{p} m \cdot 1 \equiv_{p} m$,

and

$c^d \equiv_{q} m^1 \cdot (m^{q - 1})^{k(p - 1)} \equiv_{q} m \cdot (1)^{k(p - 1)} \equiv_{q} m \cdot 1 \equiv_{q} m$.

Finally, that consequence of the Chinese Remainder Theorem guarantees what we need, namely that since $c^d \equiv_{p} m$ and $c^d \equiv_{q} m$, then

$c^d \equiv_{pq} m$, or $c^d \equiv_{n} m$ which, because $0 \le m < n$, means $m$ is the unique solution.



### Concretize

Get your hands dirty with $p = 23$ and $q = 29$:

In [1]:
def igcd(b, n):
    x0, x1, y0, y1 = 1, 0, 0, 1
    while n != 0:
        q, b, n = b // n, n, b % n
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1
    return b, x0, y0

def TUMMI(e, t):
    hcf, x, y = igcd(e, t)
    if hcf != 1: # hcf = highest common factor
        raise ValueError(f"TUMMI does not exist for e = {e} and t = {t}.")
    return x % t #  (mod t) of e

p = 23 # prime number #1
q = 29 # prime number #2
n = p * q # product of p and q
t = (p - 1) * (q - 1) # totient
e = 3 # the encryption exponent
d = TUMMI(e, t) # the decryption exponent
de = d * e # product of the encryption and decryption
k = (de - 1) // t # What is the purpose of k?

print({'p':p, 'q':q, 'n':n, 't':t, 'e':e, 'd':d, 'de':de, 'k':k})

{'p': 23, 'q': 29, 'n': 667, 't': 616, 'e': 3, 'd': 411, 'de': 1233, 'k': 2}


#### Alternate

What other types of encryption are there besides RSA?

1. Triple DES: Triple Data Encryption Algorithm or Triple-DES uses symmetric encryption. It is an advanced version of DES block cipher, which used to have a 56-bit key. However, and as the name suggests, TDES encrypts data using 56-bit key thrice, making it a 168-bit key. 
2. AES: The Advanced Encryption Standard (AES) is also symmetric encryption based on the Rijndael algorithm.  It uses block-cipher and encrypts one fixed-size block at a time. It works in 128-bit or 192-bit but can be extended up to the 256-bit key length. To encrypt every bit, there are different rounds. For instance, 128-bit will have 10 rounds, 192-bit will have 12 rounds and so on
It is considered as one of the best encryption algorithms because it was developed by the US National Institute of Standards and Technology. It is also one of the secured types of encryptions as it works on a single private key.
3. Blowfish: Another encryption algorithm designed to replace DES, Blowfish is a symmetric block cipher, which works on a variable key length from 32 bits to 448 bits. Since it is a block cipher, therefore it divides data or a message into fixed 64-bit blocks when encrypting and decrypting. 
4. Twofish: Also a symmetric block cipher, Twofish is an advanced version of Blowfish encryption. It has a block size of 128-bits and can extend to the 256-bit key length. Like other symmetric ciphers, it also breaks data into fixed-length blocks. However, it functions in 16 rounds regardless of how large the data is. Amongst the various types of encryption, this one is flexible. It allows you to choose the encryption process to be quick while the key setup to be slow and vice versa.
5. FPE: Format Preserving Encryption (FPE) is a fairly new encryption method. It encrypts your data in a similar format. For instance, if you have encrypted your password having 6 letters, 5 numbers and 4 special letters, then your output will be a different combination of a similar format.
In other words, if you use this encryption technique, it will preserve the format of your plain text that is after encryption the structure of your data will remain the same.
It is widely used in financial database systems, banking systems, retail, etc.

Source: https://www.goodcore.co.uk/blog/types-of-encryption/

##### Answer

Learning this is a GPAO of gargantuan proportions!

Start small with a site like https://www.di-mgt.com.au/rsa_alg.html.


###### Favorite

What is your favorite resource that you think explains RSA the best?

When you have one, enter a link to it with a brief description in the outside chat channel.

This is a typical Wikipedia source on the typical RSA algorithm, but I like how it breaks down each setup and explains the meaning and purpose of each variable used throughout the process. 

(Favorite RSA Source: https://simple.wikipedia.org/wiki/RSA_algorithm) 